# 🎮 Grid World Basics - หุ่นยนต์เรียนรู้หาทาง

## 🤖 เรื่องราวของ Agent ตัวแรก

ลองจินตนาการว่าคุณสร้างหุ่นยนต์ตัวหนึ่ง วางไว้ในห้องขนาด 5×5 ช่อง

**เป้าหมาย:** เดินไปหาประตูทางออก 🚪

**อุปสรรค:**
- มีหลุมพัง (ห้ามตก!) ❌
- ไม่รู้ว่าประตูอยู่ไหน 🤔

**กฎของเกม:**
- ถึงประตู → รางวัล +10 🎉
- ตกหลุม → ลงโทษ -10 😱
- เดินทุกก้าว → -1 (เพื่อหาทางสั้นสุด)

มาสร้าง Grid World แรกของเรากันเลย! 🚀

In [ ]:
# Import libraries
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import seaborn as sns
from IPython.display import clear_output
import time

print("✅ Libraries loaded!")

---

## 🏗️ Part 1: สร้าง Grid World Environment

In [ ]:
class GridWorld:
    """
    Grid World Environment สำหรับ Reinforcement Learning
    
    Grid Layout (5x5):
    S = Start (0,0)
    G = Goal (4,4)
    X = Hole (obstacles)
    . = Empty space
    """
    
    def __init__(self, size=5):
        self.size = size
        self.start_pos = (0, 0)
        self.goal_pos = (size-1, size-1)
        
        # กำหนดตำแหน่งหลุม
        self.holes = [(1, 1), (1, 3), (3, 1)]
        
        # Current state
        self.agent_pos = self.start_pos
        
        # Actions: 0=Up, 1=Down, 2=Left, 3=Right
        self.actions = ['↑', '↓', '←', '→']
        self.n_actions = len(self.actions)
        
    def reset(self):
        """รีเซ็ต environment กลับไปเริ่มต้น"""
        self.agent_pos = self.start_pos
        return self.agent_pos
    
    def step(self, action):
        """
        ทำ action และคืนค่า (next_state, reward, done)
        
        Args:
            action: 0=Up, 1=Down, 2=Left, 3=Right
            
        Returns:
            next_state: ตำแหน่งใหม่
            reward: รางวัลที่ได้รับ
            done: จบ episode หรือยัง
        """
        row, col = self.agent_pos
        
        # คำนวณตำแหน่งใหม่
        if action == 0:  # Up
            new_pos = (max(row - 1, 0), col)
        elif action == 1:  # Down
            new_pos = (min(row + 1, self.size - 1), col)
        elif action == 2:  # Left
            new_pos = (row, max(col - 1, 0))
        elif action == 3:  # Right
            new_pos = (row, min(col + 1, self.size - 1))
        
        self.agent_pos = new_pos
        
        # คำนวณ reward
        if new_pos == self.goal_pos:
            reward = 10  # ถึงเป้าหมาย!
            done = True
        elif new_pos in self.holes:
            reward = -10  # ตกหลุม!
            done = True
        else:
            reward = -1  # penalty เวลา
            done = False
        
        return new_pos, reward, done
    
    def render(self):
        """แสดงภาพ Grid World"""
        fig, ax = plt.subplots(figsize=(6, 6))
        
        # วาดกริด
        for i in range(self.size):
            for j in range(self.size):
                color = 'white'
                
                # สีของแต่ละช่อง
                if (i, j) == self.start_pos:
                    color = 'lightgreen'
                    label = 'S'
                elif (i, j) == self.goal_pos:
                    color = 'gold'
                    label = 'G'
                elif (i, j) in self.holes:
                    color = 'red'
                    label = 'X'
                elif (i, j) == self.agent_pos:
                    color = 'lightblue'
                    label = '🤖'
                else:
                    label = ''
                
                # วาดสี่เหลี่ยม
                rect = Rectangle((j, self.size-1-i), 1, 1, 
                                facecolor=color, edgecolor='black', linewidth=2)
                ax.add_patch(rect)
                
                # ใส่ label
                ax.text(j+0.5, self.size-0.5-i, label, 
                       ha='center', va='center', fontsize=20, fontweight='bold')
        
        ax.set_xlim(0, self.size)
        ax.set_ylim(0, self.size)
        ax.set_aspect('equal')
        ax.axis('off')
        
        plt.title('Grid World', fontsize=16, fontweight='bold')
        plt.tight_layout()
        plt.show()

# สร้าง environment
env = GridWorld(size=5)
print("✅ Grid World created!")
print(f"   Size: {env.size}×{env.size}")
print(f"   Start: {env.start_pos}")
print(f"   Goal: {env.goal_pos}")
print(f"   Holes: {env.holes}")
print(f"   Actions: {env.actions}")

In [ ]:
# แสดง Grid World
env.render()

---

## 🎲 Part 2: Random Agent - วิ่งสุ่มไปเรื่อยๆ

In [ ]:
def random_agent(env, max_steps=50):
    """
    Agent ที่เลือก action แบบสุ่ม
    """
    state = env.reset()
    total_reward = 0
    trajectory = [state]
    
    print("🎲 Random Agent starts!\n")
    
    for step in range(max_steps):
        # เลือก action แบบสุ่ม
        action = np.random.randint(0, env.n_actions)
        
        # ทำ action
        next_state, reward, done = env.step(action)
        
        # บันทึก
        trajectory.append(next_state)
        total_reward += reward
        
        print(f"Step {step+1}: {state} → {env.actions[action]} → {next_state}, Reward: {reward:+3.0f}")
        
        if done:
            if next_state == env.goal_pos:
                print(f"\n🎉 SUCCESS! Reached goal in {step+1} steps!")
            else:
                print(f"\n😱 FAILED! Fell into hole at {next_state}")
            break
        
        state = next_state
    else:
        print(f"\n⏱️ TIME OUT! Didn't reach goal in {max_steps} steps")
    
    print(f"\n📊 Total Reward: {total_reward}")
    print(f"📍 Trajectory: {' → '.join([str(s) for s in trajectory])}")
    
    return total_reward, trajectory

# ทดลอง Random Agent
reward, path = random_agent(env)

In [ ]:
# ทดลองหลายๆ ครั้ง
print("🔬 Running 100 episodes with Random Agent...\n")

successes = 0
failures = 0
timeouts = 0
rewards = []

for episode in range(100):
    env.reset()
    total_reward = 0
    
    for step in range(50):
        action = np.random.randint(0, env.n_actions)
        next_state, reward, done = env.step(action)
        total_reward += reward
        
        if done:
            if next_state == env.goal_pos:
                successes += 1
            else:
                failures += 1
            break
    else:
        timeouts += 1
    
    rewards.append(total_reward)

# สถิติ
print("📊 Statistics (100 episodes):")
print(f"   ✅ Success: {successes} ({successes}%)")
print(f"   ❌ Failed (holes): {failures} ({failures}%)")
print(f"   ⏱️ Timeout: {timeouts} ({timeouts}%)")
print(f"   📈 Average Reward: {np.mean(rewards):.2f}")
print(f"   🏆 Best Reward: {np.max(rewards)}")
print(f"   😢 Worst Reward: {np.min(rewards)}")

# Plot
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.bar(['Success', 'Failed', 'Timeout'], [successes, failures, timeouts], 
        color=['green', 'red', 'orange'])
plt.title('Random Agent Performance', fontsize=13, fontweight='bold')
plt.ylabel('Count')
plt.grid(axis='y', alpha=0.3)

plt.subplot(1, 2, 2)
plt.hist(rewards, bins=20, color='skyblue', edgecolor='black')
plt.axvline(np.mean(rewards), color='red', linestyle='--', linewidth=2, label=f'Mean: {np.mean(rewards):.1f}')
plt.title('Reward Distribution', fontsize=13, fontweight='bold')
plt.xlabel('Total Reward')
plt.ylabel('Frequency')
plt.legend()
plt.grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("\n💡 Observation: Random Agent ไม่ค่อยสำเร็จเลย!")
print("   → ต้องมีวิธีที่ฉลาดกว่านี้...")

---

## 🧠 Part 3: Rule-based Agent - มีกฎเกณฑ์

สร้าง Agent ที่มีกฎง่ายๆ: **"เดินไปทางขวาและลง"**

In [ ]:
def rule_based_agent(env, max_steps=50):
    """
    Agent ที่มีกฎ: ถ้าเป็นไปได้ → เดินขวาหรือลง
    (เพราะ Goal อยู่มุมขวาล่าง)
    """
    state = env.reset()
    total_reward = 0
    trajectory = [state]
    
    print("🧠 Rule-based Agent starts!\n")
    print("   Strategy: Prefer Right and Down\n")
    
    for step in range(max_steps):
        row, col = state
        
        # กฎ: เลือก Right หรือ Down (ไปหา Goal)
        if col < env.size - 1 and row < env.size - 1:
            # เลือก Right หรือ Down แบบสุ่ม
            action = np.random.choice([1, 3])  # Down or Right
        elif col < env.size - 1:
            action = 3  # Right
        elif row < env.size - 1:
            action = 1  # Down
        else:
            action = np.random.randint(0, env.n_actions)  # Random
        
        # ทำ action
        next_state, reward, done = env.step(action)
        
        # บันทึก
        trajectory.append(next_state)
        total_reward += reward
        
        print(f"Step {step+1}: {state} → {env.actions[action]} → {next_state}, Reward: {reward:+3.0f}")
        
        if done:
            if next_state == env.goal_pos:
                print(f"\n🎉 SUCCESS! Reached goal in {step+1} steps!")
            else:
                print(f"\n😱 FAILED! Fell into hole at {next_state}")
            break
        
        state = next_state
    else:
        print(f"\n⏱️ TIME OUT!")
    
    print(f"\n📊 Total Reward: {total_reward}")
    
    return total_reward, trajectory

# ทดลอง
reward, path = rule_based_agent(env)

---

## 📊 Part 4: เปรียบเทียบ Random vs Rule-based

In [ ]:
# ทดสอบทั้ง 2 แบบ
n_episodes = 100

# Random Agent
random_rewards = []
random_success = 0

for _ in range(n_episodes):
    env.reset()
    total_reward = 0
    
    for step in range(50):
        action = np.random.randint(0, env.n_actions)
        next_state, reward, done = env.step(action)
        total_reward += reward
        
        if done:
            if next_state == env.goal_pos:
                random_success += 1
            break
    
    random_rewards.append(total_reward)

# Rule-based Agent  
rule_rewards = []
rule_success = 0

for _ in range(n_episodes):
    env.reset()
    total_reward = 0
    
    for step in range(50):
        row, col = env.agent_pos
        
        if col < env.size - 1 and row < env.size - 1:
            action = np.random.choice([1, 3])
        elif col < env.size - 1:
            action = 3
        elif row < env.size - 1:
            action = 1
        else:
            action = np.random.randint(0, env.n_actions)
        
        next_state, reward, done = env.step(action)
        total_reward += reward
        
        if done:
            if next_state == env.goal_pos:
                rule_success += 1
            break
    
    rule_rewards.append(total_reward)

# เปรียบเทียบ
print("📊 Comparison (100 episodes each):\n")
print("="*60)

print(f"\n🎲 Random Agent:")
print(f"   Success Rate: {random_success}%")
print(f"   Avg Reward: {np.mean(random_rewards):.2f}")

print(f"\n🧠 Rule-based Agent:")
print(f"   Success Rate: {rule_success}%")
print(f"   Avg Reward: {np.mean(rule_rewards):.2f}")

print(f"\n💡 Improvement: {rule_success - random_success}% higher success rate!")

# Visualization
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Success Rate
axes[0].bar(['Random', 'Rule-based'], [random_success, rule_success], 
           color=['red', 'green'], alpha=0.7)
axes[0].set_ylabel('Success Rate (%)')
axes[0].set_title('Success Rate Comparison', fontsize=13, fontweight='bold')
axes[0].grid(axis='y', alpha=0.3)

# Reward Distribution
axes[1].hist(random_rewards, bins=20, alpha=0.5, label='Random', color='red')
axes[1].hist(rule_rewards, bins=20, alpha=0.5, label='Rule-based', color='green')
axes[1].axvline(np.mean(random_rewards), color='red', linestyle='--', linewidth=2)
axes[1].axvline(np.mean(rule_rewards), color='green', linestyle='--', linewidth=2)
axes[1].set_xlabel('Total Reward')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Reward Distribution', fontsize=13, fontweight='bold')
axes[1].legend()
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

---

## 🎓 สรุปบทเรียน

### ✅ สิ่งที่เราได้เรียนรู้:

1. **Grid World Environment**
   - State, Action, Reward
   - Terminal states (Goal, Hole)
   - Episode และ Trajectory

2. **Random Agent**
   - เลือก action แบบสุ่ม
   - Success rate ต่ำมาก (~10-20%)
   - ไม่มีการเรียนรู้

3. **Rule-based Agent**
   - มีกฎง่ายๆ (เดินขวา/ลง)
   - Success rate สูงกว่า (~30-50%)
   - แต่ยังไม่ได้เรียนรู้จากประสบการณ์

### 🤔 คำถาม:

**จะทำยังไงให้ Agent เรียนรู้เองได้?**

→ ตอบ: ใช้ **Q-Learning**!

### 🚀 Next Step:

👉 [02_mdp_concepts.ipynb](02_mdp_concepts.ipynb)

เรียนรู้ Markov Decision Process และ Bellman Equation!